# Historical Training Data Pipeline

### READ ME

* This notebook is to be ran at the start of every year to collect the most recent data for testing
* In College sports, teams can enter and leave a conference. Instead of trying to manage of list of school movements this pipeline will look at the conference members for the current year and use those current members to fetch the historical data for.
* This notebook should be executed in sequential order, block by block

### Dependencies

In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import random
import random as rnd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statistics
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import time
from tqdm import tqdm
from datetime import date
from pandasql import sqldf
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

### User Agent Information

In [2]:
software_names = [SoftwareName.FIREFOX.value]
operating_systems = [OperatingSystem.MAC.value, OperatingSystem.MAC.value]
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=30)
user_agent = user_agent_rotator.get_random_user_agent()

In [3]:
url = 'https://www.sports-reference.com/cbb/schools/virginia/men/2019-gamelogs-advanced.html'
data = pd.read_html(url)
df = data[0]

### Fetching Team Member Link Structure

In [4]:
# URL takes us to the Conference team home page
url = 'https://www.sports-reference.com/cbb/conferences/acc/men/schools.html'
headers = {"User-Agent":user_agent}
    #Initiating the web page and finding table contents
html = requests.get(url,headers=headers)
soup = BeautifulSoup(html.text, 'lxml')

linkslist = soup.findAll('a')

In [5]:
linkslistUpdated = []

In [6]:
for link in linkslist:
    if '/cbb/schools/' in str(link):
        if '/men/' in str(link):
            if '.' not in str(link):
                cleanedString = str(link).replace('"','*')
                cleanedString = cleanedString.split('*')[1]
                linkslistUpdated.append(cleanedString)

In [14]:
linkslistUpdated

['/cbb/schools/north-carolina/men/',
 '/cbb/schools/duke/men/',
 '/cbb/schools/north-carolina-state/men/',
 '/cbb/schools/virginia/men/',
 '/cbb/schools/wake-forest/men/',
 '/cbb/schools/maryland/men/',
 '/cbb/schools/clemson/men/',
 '/cbb/schools/georgia-tech/men/',
 '/cbb/schools/florida-state/men/',
 '/cbb/schools/miami-fl/men/',
 '/cbb/schools/virginia-tech/men/',
 '/cbb/schools/boston-college/men/',
 '/cbb/schools/syracuse/men/',
 '/cbb/schools/south-carolina/men/',
 '/cbb/schools/notre-dame/men/',
 '/cbb/schools/pittsburgh/men/',
 '/cbb/schools/louisville/men/',
 '/cbb/schools/southern-methodist/men/',
 '/cbb/schools/stanford/men/',
 '/cbb/schools/california/men/',
 '/cbb/schools/boston-university/men/',
 '/cbb/schools/houston/men/',
 '/cbb/schools/northwestern/men/']

### Regualr & Advanced Box Score Gamelog Functions

In [7]:
def regularBoxScoreScraper(team,season):  
    '''
    INPUTS
    team - (string input) college basketball team abbreviation, This should come from the TeamAbbreviation notebook
    season - (string input) This is a four character parameter yyyy
    ----------------------------------------------------------------
    OUTPUT
    df - this data frame contains advanced box score game logs
    '''
    #Creating our url string to go fetch website data
    url = 'https://www.sports-reference.com' + team + season+'-gamelogs.html'
    data = pd.read_html(url)
    df = data[0]
   # https://www.sports-reference.com/cbb/schools/north-carolina/men/2024-gamelogs.html
    #calling our user agent variables to create a request header
    #headers = {"User-Agent":user_agent}
    #Initiating the web page and finding table contents
   # html = requests.get(url,headers=headers)
   # soup = BeautifulSoup(html.text, 'lxml')
   # table = soup.find('div', {'class':'table_wrapper'})
    #table_rows = table.find_all('tr')
   # df = pd.DataFrame(pd.read_html(str(table))[0])
    df['Date'] =df[('Unnamed: 1_level_0',                'Date')]
    df['Location'] = df[('Unnamed: 2_level_0',                'Unnamed: 2_level_1')]
    df['Opposition'] =df[('Unnamed: 3_level_0',                'Opp')]
    df['Team_Points'] =df[('Unnamed: 5_level_0',                'Tm')]
    df['Opponent_Points'] =df[('Unnamed: 6_level_0',                'Opp')]
    df['FTA']=df[('School',                'FTA')]
    df['FT_Perc']=df[('School',                'FT%')]
    df['FG']=df[('School',                'FG')]
    df['FGA_RAW']=df[('School',                'FGA')]
    df['ThreeP']=df[('School',                '3P')]
    df['ThreePA']=df[('School',                '3PA')]
    df['ThreePA_Perc']=df[('School',                '3P%')]
    df['Offensive_Rebounds']=df[('School',                'ORB')]
    df['Total_Rebounds']=df[('School',                'TRB')]
    df['Assists']=df[('School',                'AST')]
    df['Steals']=df[('School',                'STL')]
    df['Blocks']=df[('School',                'BLK')]
    df['TurnOvers']=df[('School',                'TOV')]
    df['Personal_Fouls']=df[('School',                'PF')]
    #Elimiating the fake column header row
    df.columns = df.columns.droplevel(1)
    #The next list is making sure we are only keeping the data fields we care about
    df = df[[

            'Date',
            'Location',
            'Opposition',
            'Team_Points',
            'Opponent_Points',
            'FTA',
            'FT_Perc',
            'FG',
            'FGA_RAW',
            'ThreeP',
            'ThreePA',
            'ThreePA_Perc',
            'Offensive_Rebounds',
            'Total_Rebounds',
            'Assists',
            'Steals',
            'Blocks',
            'TurnOvers',
            'Personal_Fouls'
            ]]
    #Renamib the unnamed column headers
    df.rename(columns={ 'Opposition':'Opponent'
                        
                       }, inplace = True
              )
    df = df[df['Opponent'].notnull()]
    df = df[df.Opponent != 'Opp']
    df['Team'] = str(team).split('/')[3]
    df['Season'] = str(years)
    #df['Points_Scored'] = df['Team_Points'].astype(int) + df['Opponent_Points'].astype(int)
    #df['FGA'] = df['FGA_RAW'].astype(int) - df['ThreePA'].astype(int)
    
    #df['FG_Perc'] = (df['FG'].astype(int) - df['ThreeP'].astype(int))/df['FGA'].astype(int)
    
    df["Location"].fillna("VS", inplace = True)
    df = df[[

            'Date',
            'Location',
            'Team',
            'Season',
            'Opponent',
            'Team_Points',
            'Opponent_Points',
            #'Points_Scored',
            'FTA',
            'FT_Perc',
            'FG',
            #'FGA',
            #'FG_Perc',
            'ThreeP',
            'ThreePA',
            'ThreePA_Perc',
            'Offensive_Rebounds',
            'Total_Rebounds',
            'Assists',
            'Steals',
            'Blocks',
            'TurnOvers',
            'Personal_Fouls'
            ]]
    return df

In [8]:
def advancedBoxScoreScraper(team,season):  
    '''
    INPUTS
    team - (string input) college basketball team abbreviation, This should come from the TeamAbbreviation notebook
    season - (string input) This is a four character parameter yyyy
    ----------------------------------------------------------------
    OUTPUT
    df - this data frame contains advanced box score game logs
    '''
    #Creating our url string to go fetch website data
    url = 'https://www.sports-reference.com' + team + season+'-gamelogs-advanced.html'
    data = pd.read_html(url)
    df = data[0]
    #calling our user agent variables to create a request header
    #headers = {"User-Agent":user_agent}
    #Initiating the web page and finding table contents
    #html = requests.get(url,headers=headers)
    #soup = BeautifulSoup(html.text, 'lxml')
    #table = soup.find('div', {'class':'table_wrapper'})
    #table_rows = table.find_all('tr')
    #df = pd.DataFrame(pd.read_html(str(table))[0])
    #Creating our feature based on the offensive/defensive four factors categories
    df['Offensive_EFG']=df[('Offensive Four Factors',                'eFG%')]
    df['Defensive_EFG']=df[('Defensive Four Factors',                'eFG%')]
    df['Offensive_TurnOver_Rate'] = df[('Offensive Four Factors',                'TOV%')]
    df['Defensive_TurnOver_Rate'] = df[('Defensive Four Factors',                'TOV%')]
    df['Offensive_Rebounding_Rate'] = df[('Offensive Four Factors',                'ORB%')]
    df['Defensive_Rebounding_Rate'] = df[('Defensive Four Factors',                'DRB%')]
    #Creating our features based on the advanced categories
    df['OffensiveRating']=df[('Unnamed: 7_level_0',                'ORtg')]
    df['DefensiveRating']=df[('Unnamed: 8_level_0',                'DRtg')]
    df['Pace']=df[('Unnamed: 9_level_0',                'Pace')]
    df['FreeThrowAttemptRate']=df[('Unnamed: 10_level_0',                'FTr')]
    df['ThreePointAttemptRate']=df[('Unnamed: 11_level_0',                '3PAr')]
    df['TrueShootingRate']=df[('Unnamed: 12_level_0',                'TS%')]
    #Elimiating the fake column header row
    df.columns = df.columns.droplevel(1)
    #Renamib the unnamed column headers
    df.rename(columns={ 'Unnamed: 0_level_0': "Game"
                       ,'Unnamed: 1_level_0': "Date"
                       ,'Unnamed: 2_level_0': "Location"
                        ,'Unnamed: 3_level_0':'Opponent'
                       ,'Unnamed: 4_level_0':'Result'
                        ,'Unnamed: 5_level_0':"Team_Score"
                       ,'Unnamed: 6_level_0':"Opponent_Score"
                       

                       }, inplace = True
              )
    #The target table is defaulted to be broken out into pieces so every x rows, the column headers re appear
    # th next two lines remove this fake column header collumns
    df = df[df['Opponent'].notnull()]
    df = df[df.Opponent != 'Opp']
    df['Team'] =  str(team).split('/')[3]
    df['Season'] = str(years)
    #The next list is making sure we are only keeping the data fields we care about
    df = df[[
            'Game',
            'Date',
            'Location',
            'Team',
            'Season',
            'Opponent',
            'Result',
            'Team_Score',
            'Opponent_Score',
            'Offensive_EFG',
            'Defensive_EFG',
            'Offensive_TurnOver_Rate',
            'Defensive_TurnOver_Rate',
            'Offensive_Rebounding_Rate',
            'Defensive_Rebounding_Rate',
            'OffensiveRating',
            'DefensiveRating',
            'Pace',
            'FreeThrowAttemptRate',
            'ThreePointAttemptRate',
            'TrueShootingRate'
            ]]
    df["Location"].fillna("VS", inplace = True)
    return df

### Data Extraction Process

In [9]:
yearList = ['2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']

In [10]:
regularTeamBoxScoreFrame = pd.DataFrame()

In [11]:
# Loading historical Data for the regular box score gamelogs
for team in tqdm(linkslistUpdated):
    for years in yearList:
        boxscoreDF = regularBoxScoreScraper(team,years)
        regularTeamBoxScoreFrame = regularTeamBoxScoreFrame.append(boxscoreDF)
        time.sleep(3.3)

100%|██████████| 23/23 [13:25<00:00, 35.04s/it]


In [12]:
regularTeamBoxScoreFrame.head()

,Date,Location,Team,Season,Opponent,Team_Points,Opponent_Points,FTA,FT_Perc,FG,ThreeP,ThreePA,ThreePA_Perc,Offensive_Rebounds,Total_Rebounds,Assists,Steals,Blocks,TurnOvers,Personal_Fouls
0,2014-11-14,VS,north-carolina,2015,North Carolina Central,76,60,30,.667,27,2,11,.182,14,47,15,8,10,14,24
1,2014-11-16,VS,north-carolina,2015,Robert Morris,103,59,36,.750,36,4,16,.250,18,45,29,9,5,8,20
2,2014-11-22,N,north-carolina,2015,Davidson,90,72,28,.643,33,6,17,.353,13,42,21,4,9,9,16
3,2014-11-26,N,north-carolina,2015,Butler,66,74,32,.563,22,4,16,.250,12,35,14,9,10,19,24
4,2014-11-27,N,north-carolina,2015,UCLA,78,56,19,.737,28,8,23,.348,11,31,15,13,0,11,25


In [13]:
regularTeamBoxScoreFrame.shape

(7527, 20)

In [23]:
advancedTeamBoxScoreFrame = pd.DataFrame()

In [24]:
# Loading historical Data for the regular box score gamelogs
for team in tqdm(linkslistUpdated):
    for years in yearList:
        boxscoreDF = advancedBoxScoreScraper(team,years)
        advancedTeamBoxScoreFrame = advancedTeamBoxScoreFrame.append(boxscoreDF)
        time.sleep(3.3)

100%|██████████| 23/23 [13:18<00:00, 34.72s/it]


In [25]:
advancedTeamBoxScoreFrame.head()

,Game,Date,Location,Team,Season,Opponent,Result,Team_Score,Opponent_Score,Offensive_EFG,...,Offensive_TurnOver_Rate,Defensive_TurnOver_Rate,Offensive_Rebounding_Rate,Defensive_Rebounding_Rate,OffensiveRating,DefensiveRating,Pace,FreeThrowAttemptRate,ThreePointAttemptRate,TrueShootingRate
0,1,2014-11-14,VS,north-carolina,2015,North Carolina Central,W,76,60,.459,...,15.7,16.7,45.2,76.7,102.0,80.5,74.5,.492,.180,.505
1,2,2014-11-16,VS,north-carolina,2015,Robert Morris,W,103,59,.535,...,8.3,21.0,54.5,77.1,132.3,75.8,77.9,.507,.225,.585
2,3,2014-11-22,N,north-carolina,2015,Davidson,W,90,72,.522,...,9.9,11.1,36.1,69.0,115.8,92.6,77.7,.406,.246,.547
3,4,2014-11-26,N,north-carolina,2015,Butler,L,66,74,.421,...,20.8,18.1,31.6,51.1,81.5,91.4,81.0,.561,.281,.457
4,5,2014-11-27,N,north-carolina,2015,UCLA,W,78,56,.478,...,12.6,27.8,28.9,69.0,104.2,74.8,74.9,.284,.343,.513


In [134]:
advancedTeamBoxScoreFrame.shape

(7527, 20)

### Data Save Process

In [26]:
# Storing the regular box score data to an excel sheet
regularTeamBoxScoreFrame.to_excel("regularTeamBoxScoreGameLogHistory.xlsx")

In [27]:
# Storing the advanced box score data to an excel sheet
advancedTeamBoxScoreFrame.to_excel("advancedTeamBoxScoreGameLogHistory.xlsx")

In [31]:
# Combining the two datasets to get the regular and advanced data on one sheet
# Using a pandasql library to perform the data join
run_query = lambda query: sqldf(query, globals())

query0 = '''
SELECT
rg.Date,
rg.Location,
rg.Team,
rg.Season,
rg.Opponent,
rg.Team_Points,
rg.Opponent_Points,
rg.FTA,
rg.FT_Perc,
rg.FG,
rg.ThreeP,
rg.ThreePA,
rg.ThreePA_Perc,
rg.Offensive_Rebounds,
rg.Total_Rebounds,
rg.Assists,
rg.Steals,
rg.Blocks,
rg.TurnOvers,
rg.Personal_Fouls,
ag.Offensive_EFG,
ag.Defensive_EFG,
ag.Offensive_TurnOver_Rate,
ag.Defensive_TurnOver_Rate,
ag.Offensive_Rebounding_Rate,
ag.Defensive_Rebounding_Rate,
ag.OffensiveRating,
ag.DefensiveRating,
ag.Pace,
ag.FreeThrowAttemptRate,
ag.ThreePointAttemptRate,
ag.TrueShootingRate
FROM regularTeamBoxScoreFrame rg
LEFT JOIN advancedTeamBoxScoreFrame ag
on rg.Date = ag.Date
and rg.Team = ag.Team

'''

In [32]:
# Executing the query stated above
result_0 = run_query(query0)
result_0.shape

(7529, 32)

In [33]:
result_0.to_excel("combinedTeamBoxScoreGameLogHistory.xlsx")